# Fájlkezelés pythonban

A python elég régóta létezik, így a történelme során elég sokféle fájl-kezelési megoldás alakult ki. Mi viszont csak egyetlen, modern megoldással fogunk foglalkozni a Pathlibbel!

## Path objektum

A modern, ajánlott megoldás fájlokra/könyvtárakra a pathlib.Path objektum. Ez jelképezi számunkra az adott könyvtárat. A kölönféle operációs rendszerek eltéréseivel sem kell foglalkoznunk (egyes ósdi operációs rendszerek nem átalanak `\` jelet használni útvonalelválasztóként), mivel a Path objetum felüldefiniálja az osztás-jelet így egyszerűen csak azt használjuk.

In [ ]:
# beimportáljuk a Path objektumot
from pathlib import Path

# aktuális munkakönyvtár
cwd = Path.cwd()
print("Itt vagyunk most:", cwd)

# saját home könyvtár
home = Path.home()
print("Ez a felhasználónk saját könyvtára:", home)

# konkrét elérési út
p = Path("adatok/meresek.txt")

# útvonalak összefűzése: / operátorral
adat_dir = Path("adatok")
fajl = adat_dir / "meresek.txt"

fajl # A típusa most PosixPath lesz mert linux alatt vagyunk
# ha windows alatt műveled ugyanezt, akkor WindowsPath lesz


A pathlib segítségével gyorsan tudjuk kezelni a fájlokat. Többek között megnézhetjük, hogy:
```python
p.exists()      # létezik?
p.is_file()     # fájl?
p.is_dir()      # könyvtár?
p.name          # fájlnév kiterjesztéssel: "meresek.txt"
p.stem          # kiterjesztés nélkül: "meresek"
p.suffix        # mi a kiterjesztés: ".txt"
p.parent        # szülő könyvtára?
```

## Műveletek fájlokkal

Korábban már megnyitottunk fájlokat, de ezt megtehetjük a Path-on keresztül is. Nem is nagyon van szükségünk másra.

Írjunk ki pár adatot egy fájlba!

In [ ]:
# szuper értékes parabola adatok!
adatok = [x**2+2 for x in range(30)]

In [ ]:
from pathlib import Path

# hozzunk létre egy Path-ot (fájlt)
p = Path("kimenet/eredmeny.txt")

# ha kell, hozzd létre a (szülő) könyvtárat!
# ha létezik, az nem baj
p.parent.mkdir(parents=True, exist_ok=True)

# írd ki az adatokat (a "w" mutatja, hogy írni akarunk)
with p.open("w") as f:
    for x in adatok:
        # soremelést teszünk a végére
        f.write(f"{x}\n") # minden szám új sorba!


In [ ]:
# az fájrendszeren máris ott a fájl:
!cat kimenet/eredmeny.txt

In [ ]:
# most pedig nézzük, hogyan töltenénk vissza!

from pathlib import Path

p = Path("kimenet/eredmeny.txt")
# ha csak szövegként kell:
tartalom = p.read_text(encoding="utf-8")
print(tartalom)


In [ ]:
# vagy ha inkább egyesével olvasnánk a sorokat:
# az "r" mutatja, hogy olvasni akarunk!
with p.open("r") as f:
    for sor in f:
        print(sor.strip())


### Bináris fájlok

In [ ]:
# töltsük le a YBL logót
!curl 'https://ybl.uni-obuda.hu/wp-content/themes/yblszm/img/oe_ybl_hu.png' > ybl.png

In [ ]:
# Nem minden fájl szöveges. Ha képeket, adatokat akarunk kezelni, bináris állományt is be tudunk olvasni
p = Path("ybl.png")

adat = p.read_bytes()      # bytes
# ... feldolgozás ...
p2 = Path("logo_masolat.png")
p2.write_bytes(adat)


Azért ilyet valószínűleg elég ritkán fogunk csinálni. Ha bináris fájlokhoz nyúlunk akkor azt valószínűleg valamilyen csomaggal akarjuk feldolgozni (pl ha kép akkor a PIL csomaggal, ha adatbázis akkor az adatbáziskezel csomaggal, ha numerikus adathalmaz akkor numpy-vel) ezek pedig mind saját fájl-megnyitó függvénnyel rendelkeznek.

### Fájl átnevezés, törlés

In [ ]:
# a fájlok kezelés pofonegyszerű

# készítsünk egy útvonalat
p = Path("info.txt")

# írjunk bele valamit
p.write_text("Fontos üzenet!")

# nevezzük át
uj = Path("uj_nev.txt")
p.rename(uj)

# töröljük le, de csak ha létezik
if uj.exists():
    uj.unlink()


## Könyvtárkezelés

Bejárhatunk egy mappát és elemezhetjük benne a fájlokat, könyvtárakat:

In [ ]:
# mappa teljes bejárása
mappa = Path(".") # ez az a könyvtár ahol a program fut

for elem in mappa.iterdir():
    if elem.is_file():
        print("Fájl:", elem.name)
    elif elem.is_dir():
        print("Könyvtár:", elem.name)


De még praktikusabb, ha ismerjük a glob generátort, amivel tetszőleges fájlnévre kereshetünk, sőt akár rekurzívan is bejárhatjuk a fájlrendszert!


In [ ]:
mappa = Path("kimenet")

# Minden .txt fájl
for fajl in mappa.glob("*.txt"):
    print(fajl)            # teljes elérési út
    print(fajl.name)       # csak név


kimenet/eredmeny.txt
eredmeny.txt


A fenti példában a * jelentése: bármilyen karaktersorozat (akár üres is). Több ilyen szűrést is használhatunk!

* `*.txt` → minden valami.txt
* `adat_*.csv` → minden olyan csv, ami adat_-tal kezdődik
* `*` – 0 vagy több tetszőleges karakter
* `?` – pontosan 1 tetszőleges karakter
* `[abc]` – pontosan 1 karakter a kapcsos zárójelben felsoroltak közül

Tehát:
```python
mappa.glob("adat_??.txt")      # pl. adat_01.txt, adat_ab.txt
mappa.glob("[ab]*.log")        # 'a' vagy 'b' betűvel kezdődő .log fájlok
```

Ha minden almappában is keresni akarunk, két lehetőségünk van:

In [ ]:
gyoker = Path(".")

for fajl in gyoker.glob("**/*.txt"):
    print(fajl)


In [ ]:
# vagy:
for fajl in gyoker.rglob("*.txt"):
    print(fajl)


## Fájlkezelés

In [ ]:
# a fájlok kezelés pofonegyszerű

# készítsünk egy útvonalat
p = Path("info.txt")

# írjunk bele valamit
p.write_text("Fontos üzenet!")

# nevezzük át
uj = Path("uj_nev.txt")
p.rename(uj)

# töröljük le, de csak ha létezik
if uj.exists():
    uj.unlink()


## ZipFile - tömörített állományok kezelés

Az adatok gyakran érkeznek tömörítve. Pythonban könnyedén tudunk velük dolgozni akár anélkül, hogy mindig kibontanánk őket. Itt segít nekünk a zipfile csomag, ami a PathLibhez hasonló módon kezeli a tömörített állományokat.
